In [1]:
import librosa as lr
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import logging

from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot  


class VoiceSamples(Dataset):
    
    def __init__(self, core_name, samples_path=None, Automatic=None):
        
        self.Log = logging.getLogger()
        logging.basicConfig(level=logging.INFO)
        
        self.noiseThreshold = 1
        
        self.core_name = core_name
        self.samples_path = samples_path
        
        self.soundSamples = []
        self.sampleRate = []
        self.path = []
        
        self.chopedSamples = []
        self.chopedSr = []
        
        self.tensorMelgrams = []
        
        
        self.info = " VoiceSamples Object successfully created "
        self.Log.info(self.info)
        
        
        if Automatic:
            self.LoadSoundSamples()
            self.ChopToOneSecFragments()
            self.ChopedSignalsToTenosor()
        
    def __len__(self):
        return len(self.tensorMelgrams)
    
    def __getitem__(self, idx):
        if self.tensorMelgrams:
            return self.tensorMelgrams[idx]

    def LoadSoundSamples(self):
    
        n = 1

        while(True):
            try:
                if  self.samples_path:
                    path =  self.samples_path + self.core_name + str(n)
                else:
                    path = self.core_name + str(n)

                soundSample, sampleRate = lr.load(path)

                n += 1
                self.soundSamples.append(soundSample)
                self.sampleRate.append(sampleRate) 
                self.path.append(path)

                self.info = " Sample : " + path + " : successfully added"
                self.Log.info(self.info)

            except FileNotFoundError:
                if self.soundSamples:
                    self.info = "That's the end of database : " + str(n-1) + " : Samples added"
                    self.Log.info(self.info)
                    n = 0
                    
                    return self.soundSamples, self.sampleRate, self.path

                else:
                    self.Log.exception("Files are missing")
                    n = 0

                break

            except Exception as ex:      
                self.Log.exception("Unexpected error")
                break
        
    def getSoundSample(self, idx):
        return self.soundSamples[idx], self.sampleRate[idx]
    
    def getSoundSampleLen(self):
        try:
            if len(self.soundSamples) == len(self.sampleRate):
                return len(self.soundSamples)
            else:
                self.Log.warning("Lists: sundSamples and sampleRate are not equal!")
                
        except Exception as e:
            self.Log.exception("Unexpected error" + e)
    
    def ChopToOneSecFragments(self):
        
        # TODO: make shure user goes step by step 
        
        try:
            if len(self.soundSamples) == len(self.sampleRate):
                for idx in range(len(self.soundSamples)):
                    
                    soundSample = self.soundSamples[idx]
                    sr = self.sampleRate[idx]
                    
                    frag_max = math.trunc(len(soundSample)/float(sr))
                    step = math.trunc(sr/2);
                    last_sample = len(soundSample)

                    for frag in range(frag_max*2):
                        start = step * frag
                        stop = start + sr
                        if sr<len(soundSample):
                            if self.checkIfNotNoise(soundSample[start:stop]):
                                self.chopedSamples.append(soundSample[start:stop])
                                self.chopedSr.append(sr)
                                self.info = self.path[idx] + " : " + str(frag+1) + " : successfully choped"
                                self.Log.info(self.info)
                            else:
                                self.info = self.path[idx] + " : " + str(frag+1) + " : NOISE!"
                                self.Log.info(self.info)
                        else:
                            self.Log.warning("Something went wrong")
                            
                    if self.checkIfNotNoise(soundSample[last_sample-sr:last_sample]):
                         # incuding samples cuted by math.trunc() 
                        self.chopedSamples.append(soundSample[last_sample-sr:last_sample])
                        self.chopedSr.append(sr)
                        self.info = self.path[idx] +  " : "  + str(frag_max*2+1) + " : successfully choped"
                        self.Log.info(self.info)
                    else:
                        self.info = self.path[idx] + " : "  + str(frag+1) + " : NOISE!"
                        self.Log.info(self.info)
                
                if self.chopedSamples:
                    self.Log.info("Sucessfully choped all loaded signals and eliminated the noise!")
                    return self.chopedSamples, self.chopedSr 
                    
            else:
                self.Log.warning("Lists: sundSamples and sampleRate are not equal!")
                
        except Exception as e:
            self.e = "Unexpected error : " + str(e)
            self.Log.exception(self.e)
            
    def getChoped(self, idx):
        return self.chopedSamples[idx], self.chopedSr[idx]
        
    def getChopedLen(self):
        try:
            if len(self.chopedSamples) == len(self.chopedSr):
                    return len(self.chopedSamples)
            else:
                self.Log.warning("Lists: sundSamples and sampleRate are not equal!")
                
        except Exception as e:
            self.Log.exception("Unexpected error" + e)
            
        
    def ChopedSignalsToTenosor(self):
        
        # TODO: make shure user goes step by step 
        
        try:
        
            if len(self.chopedSamples) == len(self.chopedSr):
                for idx in range(len(self.chopedSamples)):

                    # hop length adjusted
                    STFT_signal = np.abs(lr.stft(self.chopedSamples[idx], n_fft = 512, hop_length = round(self.chopedSr[idx]/256))) 
                    STFT_signal = lr.power_to_db(STFT_signal**2,ref=np.max)

                    Melgram = STFT_signal[0:256,0:256]
                    TMelgram = torch.tensor(Melgram)
                    self.tensorMelgrams.append(TMelgram)
                    
                    self.info = " " + self.samples_path +  " : ChopedSample " + str(idx) + " : " + " : converted to tensor"
                    self.Log.info(self.info)
                
                if self.tensorMelgrams:
                    self.Log.info("Sucessfully converted all ChopedSamples to Tensors!")
                    return self.tensorMelgrams
                
            else:
                self.Log.warning("Lists: chopedSamples and chopedSr are not equal!")
                
        except Exception as e:
            self.e = "Unexpected error : " + str(e)
            self.Log.exception(self.e)
                
    
    
    def checkIfNotNoise(self, chopedSample):
    
        chopedSamplePow2 = []

        for n in range(len(chopedSample)):
            chopedSamplePow2.append(chopedSample[n]**2)
        sk = sum(chopedSamplePow2)
        if sk > self.noiseThreshold:
            return True 
        else:
            return False

In [2]:
# Use torch.save(tensor, 'file.pt') and torch.load('file.pt')

class VoiceSamplesInput():
    def __init__(self):
        
        self.vsKrystian = VoiceSamples("vsKrystian", samples_path="database/Krystian/" , Automatic=True)
        self.vsNicia = VoiceSamples("vsNicia", samples_path="database/Nicia/" , Automatic=True)

        self.targetKrystian = torch.tensor([[float(1),float(0)]])
        self.targetNicia = torch.tensor([[float(0),float(1)]])
        
        
    def __getitem__(self, idx):
        if idx % 2 == 0:
            return self.vsKrystian[int(idx/2)] ,  self.targetKrystian
        else:
            return self.vsNicia[int((idx+1)/2)] , self.targetNicia
    
    def __len__(self):
        if len(self.vsKrystian) <= len(self.vsNicia):
            return len(self.vsKrystian) * 2
        else:
            return len(self.vsNicia) * 2
            




In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class VoiceRecogModel(nn.Module):

    def __init__(self):
        super(VoiceRecogModel, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16*62*62, 1000)  # ?? from image dimension
        self.fc2 = nn.Linear(1000, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [4]:
import torch.optim as optim

# create dataset : loading and processing samples to tensors
vsInput = VoiceSamplesInput()


INFO:root: VoiceSamples Object successfully created 
/home/krys/.local/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
INFO:root: Sample : database/Krystian/vsKrystian1 : successfully added
/home/krys/.local/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
INFO:root: Sample : database/Krystian/vsKrystian2 : successfully added
/home/krys/.local/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
INFO:root:That's the end of database : 2 : Samples added
INFO:root:database/Krystian/vsKrystian1 : 1 : successfully choped
INFO:root:database/Krystian/vsKrystian1 : 2 : successfully choped
INFO:root:database/Krysti

INFO:root:database/Krystian/vsKrystian2 : 94 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 95 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 96 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 97 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 98 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 99 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 100 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 101 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 102 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 103 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 104 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 105 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 106 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 107 : NOISE!
INFO:root:database/Krystian/vsKrystian2 : 108 : successfully choped

INFO:root: database/Krystian/ : ChopedSample 73 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 74 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 75 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 76 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 77 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 78 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 79 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 80 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 81 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 82 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 83 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 84 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 85 :  : converted to tensor
INFO:root: database/Krystian/ : ChopedSample 86 :  

INFO:root:database/Nicia/vsNicia1 : 37 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 38 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 39 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 40 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 41 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 42 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 43 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 44 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 45 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 46 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 47 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 48 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 49 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 50 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 51 : successfully choped
INFO:root:database/Nicia/vsNicia1 : 52 : successfully choped
INFO:root:database/Nicia

INFO:root:database/Nicia/vsNicia2 : 94 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 95 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 96 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 97 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 98 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 99 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 100 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 101 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 102 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 103 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 104 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 105 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 106 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 107 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 108 : successfully choped
INFO:root:database/Nicia/vsNicia2 : 109 : successfully choped
INFO:root:data

INFO:root: database/Nicia/ : ChopedSample 27 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 28 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 29 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 30 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 31 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 32 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 33 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 34 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 35 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 36 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 37 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 38 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 39 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 40 :  : converted to tensor
INFO:root: database/

INFO:root: database/Nicia/ : ChopedSample 144 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 145 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 146 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 147 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 148 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 149 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 150 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 151 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 152 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 153 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 154 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 155 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 156 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 157 :  : converted to tensor
INFO:r

INFO:root: database/Nicia/ : ChopedSample 260 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 261 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 262 :  : converted to tensor
INFO:root:Sucessfully converted all ChopedSamples to Tensors!


In [5]:
# create net from VoiceRecogModel
net = VoiceRecogModel()

# loss function (using function implemented in pytorch)
criterion = nn.MSELoss()

# create your optimizer (basic optimizer)
# setting learning rate 
optimizer = optim.SGD(net.parameters(), lr=0.05)

In [6]:
import logging
logging.basicConfig(level=logging.INFO)
trainLog = logging.getLogger()

# Training Loop

# How many times learn on the same dataset
epoch = 2 
for i in range(epoch):   
    for k in range(len(vsInput)):  

        vs, target = vsInput[k]

        vs_n = (vs+80)/80
        
        optimizer.zero_grad()   # zero the gradient buffers
        input = vs_n.view(-1,1,256,256)
        output = net(input)

        loss = criterion(output, target)

        loss.backward()
        optimizer.step()    # Does the update
        
        
        info = "Training " + str(k)+"/"+str(len(vsInput))+" done"
        
        print("\n")
        print(target)
        print(output)
        print(loss)
        print(info)

INFO:root:Training 0/286 done


tensor([[1., 0.]])
tensor([[ 0.0654, -0.0859]], grad_fn=<AddmmBackward>)
tensor(0.4404, grad_fn=<MseLossBackward>)
tensor([[0., 1.]])
tensor([[ 0.1703, -0.0620]], grad_fn=<AddmmBackward>)
tensor(0.5784, grad_fn=<MseLossBackward>)


INFO:root:Training 1/286 done
INFO:root:Training 2/286 done


tensor([[1., 0.]])
tensor([[0.1909, 0.0858]], grad_fn=<AddmmBackward>)
tensor(0.3310, grad_fn=<MseLossBackward>)
tensor([[0., 1.]])
tensor([[0.3615, 0.1164]], grad_fn=<AddmmBackward>)
tensor(0.4557, grad_fn=<MseLossBackward>)


INFO:root:Training 3/286 done
INFO:root:Training 4/286 done


...

INFO:root:Training 281/286 done
INFO:root:Training 282/286 done


tensor([[1., 0.]])
tensor([[0.4790, 0.5120]], grad_fn=<AddmmBackward>)
tensor(0.2668, grad_fn=<MseLossBackward>)
tensor([[0., 1.]])
tensor([[0.5097, 0.4851]], grad_fn=<AddmmBackward>)
tensor(0.2624, grad_fn=<MseLossBackward>)


INFO:root:Training 283/286 done
INFO:root:Training 284/286 done


tensor([[1., 0.]])
tensor([[0.4876, 0.5118]], grad_fn=<AddmmBackward>)
tensor(0.2622, grad_fn=<MseLossBackward>)
tensor([[0., 1.]])
tensor([[0.5060, 0.4860]], grad_fn=<AddmmBackward>)
tensor(0.2601, grad_fn=<MseLossBackward>)


INFO:root:Training 285/286 done
